# Day 3 (Context Engineering: Sessions & Memory) 

> ⚠️ **Nota importante**  
>  A continuación se muestra un proceso secuencial inicial, a nivel de configuración, que es necesario realizar antes de empezar a trabajar con los Agentes. Se importan librerias de ADK necesarias con sus respectivos componentes, se crean las Helper Functions y las opciones de configuración de Retry.  


In [1]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [2]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Fin de Configuración

> ⚠️ **Nota importante**  
> Implementando nuestro primer agente con persistencia (Session). Puede recordar y tener conversaciones constructivas. ADK ofrece diferentes tipos de *session*, pero en este caso empezaremos por la mas sencilla: *InMemomrySessionService*  

    

In [4]:
APP_NAME = "default"
USER_ID = "default"
SESSION = "default"

MODEL_NAME = "gemini-2.5-flash-lite"



In [5]:
#Creando el Agente LLM

root_agent=Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options = retry_config),
    name="text_chat_bot",
    description="Un chat-bot de texto",
)   



In [6]:
# Paso 2: Configurar la gestión de session
# InMemorySessionService almacena las conversaciones en la RAM (temporalmente)

session_service = InMemorySessionService()



In [7]:
#Creamos el Runner

runner=Runner(agent=root_agent, app_name=APP_NAME,session_service=session_service)

print("Agente con persistencia (memoria) iniciado")
print("")
print(f"Aplicacion : {APP_NAME}")
print(f"Usuario : {USER_ID}")
print(f"Usando : {session_service.__class__.__name__}")

Agente con persistencia (memoria) iniciado

Aplicacion : default
Usuario : default
Usando : InMemorySessionService


> ⚠️ **Nota importante**
> 
> Pruebas del Agente. Ahora veremos la magia de Sessions y la persistencia temporal:  


In [9]:
#Prueba de conversación con dos consultas en la misma Session
#las dos consultas son parte de la misma SESSION, por lo tanto se mantiene el CONTEXTO


await run_session(

    runner,

    [
        "Hola!, me llamo Héctor. ¿Cual es la localidad con menos población en cataluña?",

         "Hola, ¿Cual es mi nombre?"
    ],

    "statefull-agentic-session",

)




 ### Session: statefull-agentic-session

User > Hola!, me llamo Héctor. ¿Cual es la localidad con menos población en cataluña?
gemini-2.5-flash-lite >  ¡Hola, Héctor! La localidad con menos población en Cataluña es **Espluga de Serra**, en la comarca de la Alta Ribagorça.

Según los datos más recientes, su población es muy reducida, a menudo por debajo de las 10 personas. Es un lugar con un encanto rural muy especial.

User > Hola, ¿Cual es mi nombre?
gemini-2.5-flash-lite >  Tu nombre es Héctor.


In [13]:
#nMemorySessionService es temporal. Una vez que la aplicación se detiene, todo el historial de la conversación se pierde

> ⚠️ **Nota importante**
> 
> Para que haya persistencia real, tiene que serintegrada, haciendo uso de DDBB.
> 
> Ahora haremos uso de *DatabaseSessionService*

In [8]:
#Creamos el mismo agente chat-bot de antes (Advierte que para esta creación , usamos LlmAgent)

chatbot_agent=LlmAgent(

    model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
    name="text_chat_bot",
    description="Un Chat-bot de texto con persistencia"

)
    


In [9]:
#Cambiamos a DatabaseSessionService
#La base de datos SQlite será creada automaticamente

db_url="sqlite+aiosqlite:///my_agent_data.db"  #Archivo local de SQLite
session_service=DatabaseSessionService(db_url=db_url)


In [10]:
#Creamos un nuevo Runner con persistencia storage

runner=Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("Actualizado para persistencia con base de datos")
print("")
print(f"Nombre de la base de datos: {db_url}")
print("")
print("Ahora Sessions sobrevive a los crasheos, restarts etc")

Actualizado para persistencia con base de datos

Nombre de la base de datos: sqlite+aiosqlite:///my_agent_data.db

Ahora Sessions sobrevive a los crasheos, restarts etc


> ⚠️ **Nota importante**
> 
> Vamos a proceder ahora a verificar la persistencia en DDBB. La ID de la SESSION será *test-db-session-01*

In [15]:
await run_session(
    runner,
    ["Hola, me llamo Héctor. Cual es la capital de España?","Como  me llamo?"],
    "test-db-session-01"
)




 ### Session: test-db-session-01

User > Hola, me llamo Héctor. Cual es la capital de España?
gemini-2.5-flash-lite >  Hola Héctor, la capital de España es Madrid.

User > Como  me llamo?
gemini-2.5-flash-lite >  Te llamas Héctor.


In [19]:
#Si cambiamos la SESSION, no podra acceder a los datos de la DDBB. En este caso usamos otra SESSION con ID "test-db-session-02"

await run_session(
    runner,
    ["Como me llamo?"],
    "test-db-session-02"
)



 ### Session: test-db-session-02

User > Como me llamo?
gemini-2.5-flash-lite >  Lo siento, pero como agente de IA, no tengo forma de saber tu nombre. No guardo información personal sobre los usuarios.


> ⚠️ **Nota importante**
> 
> Todo se va guardando en la BBDD, por lo tanto enseguida nos encontramos con que es enorme y empieza a bajar su rendimiento.

> ⚠️ **Nota importante**
> 
> Pero ¿y si pudiéramos resumir automáticamente el pasado? Usemos la función de Compactación de Contexto de ADK para ver cómo reducir automáticamente el contexto que se almacena en la sesión.

In [11]:
# Redefinimos nuestra app con el modo compactación

research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    
    # Esta parte es la nueva, la compactación
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Activar la compactación cada 3 invocaciones
        overlap_size=1,  # Mantener un truno previo como contexto
    ),
)

db_url = "sqlite+aiosqlite:///my_agent_data.db"  
session_service = DatabaseSessionService(db_url=db_url)

# Creamos un nuevo Runner para este Upgrade de APP
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅Aplicacion actualizada con compactación de eventos en la Base de Datos")

✅Aplicacion actualizada con compactación de eventos en la Base de Datos


C:\Users\Trabajo\AppData\Local\Temp\ipykernel_10784\1307449722.py:8: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


> ⚠️ **Nota importante**
> 
> Para todos los turnos futuros de esta conversación, el agente recibirá este **resumen** conciso en lugar del historial completo. Esto **reduce costos**, mejora el rendimiento y ayuda al agente a mantenerse enfocado en lo más importante.

> ⚠️ **Nota importante**
> 
> Por ultimo creamos un Agente con **CUSTOM TOOLS** para recuperar el **nombre de usuario** y la **nacionalidad** de una determinada **SESSION**

In [12]:
# Definir niveles de alcance para las claves de estado (siguiendo las mejores prácticas)

USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")



In [13]:
# Esto demuestra cómo las herramientas pueden ESCRIBIR en el estado de la sesión usando tool_context.

# El prefijo 'user:' indica que se trata de datos específicos del usuario.

def save_user_info(ToolContext, username: str, country: str) -> Dict[str, Any]:

    """

    Herramienta para registrar y guardar el nombre de usuario y el país en el estado de la sesión.
    
        Argumentos:
        
        user_name: El nombre de usuario que se almacenará en el estado de la sesión
        
        country: El nombre del país del usuario

    """

    # Escribir en el estado de la sesión usando el prefijo 'user:' para datos del usuario

    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}





# Esto es otra demostración de como TOOLS pueden lEER de session state

def retrieve_userinfo (tool_context:ToolContext) -> Dict[str,Any]:

    """

    Herramienta para recuperar el nombre de usuario y el país desde el estado de la sesión.


    """

    #Leer desde session state

    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")

    return {"status": "success", "user_name": user_name, "country": country}


print("Tools creadas con éxito")
     

Tools creadas con éxito


> ⚠️ **Nota importante**
> 
> Creando el Agente con las TOOLS recien codificadas arriba

In [14]:
# Configuración

APP_NAME="default"
USER_ID="default"
MODEL_NAME="gemini-2.5-flash-lite"

# Creación del Agente

root_agent = LlmAgent(
    
    model=Gemini(model=MODEL_NAME,rety_options=retry_config),

    name="text_chat_bot",

    description= """

                 Un chatbot de texto.
    
                    Herramientas para gestionar el contexto del usuario:
                    
                    Para registrar el nombre de usuario y el país cuando se proporcionen, usa la herramienta save_userinfo.
                    
                    Para obtener el nombre de usuario y el país cuando sea necesario, usa la herramienta retrieve_userinfo.


                 """,
    
    tools = [save_user_info, retrieve_userinfo],
)


# Iniciando SESSION SERVICE y RUNNER

session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")

print("Agente con TOOLS en SESSION inciado")



Agente con TOOLS en SESSION inciado


> ⚠️ **Nota importante**
>
> Probando como el agente usa SESSION STATE para recuperar la información a través de la conversación

In [16]:

await run_session(
    runner,
    [
        "Hola, que harás hoy?, Cual es mi nombre?",  # El agente aquí no debería conocer mi nombre
        "Me llamo Héctor, soy de España",  # Aqui se da el nombre y el país, el Agente debería recoger esta información
        "Como me llamo? De donde soy?",  # El Agente aqui debería recuper la información
    ],
    "state-demo-session",
)


 ### Session: state-demo-session

User > Hola, que harás hoy?, Cual es mi nombre?
gemini-2.5-flash-lite >  ¡Hola! Estoy aquí para ayudarte con lo que necesites. Aún no sé tu nombre, ¿te gustaría decírmelo?

User > Me llamo Héctor, soy de España
gemini-2.5-flash-lite >  Hola Héctor, ¡encantado de conocerte! A partir de ahora te recordaré. ¿Hay algo más en lo que pueda ayudarte hoy?

User > Como me llamo? De donde soy?


> ⚠️ **Nota importante**
>
> Inspección del estado de la sesión
>
> Inspeccionemos directamente el estado de la sesión para ver qué se ha almacenado:


In [17]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session"
)

print("SESSION STATE contiene:")
print(session.state)


SESSION STATE contiene:
{}


> ⚠️ **Nota importante**
>
> Por ultimo limpiamos la DATABASE (Opcional)

In [1]:
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Base de Datos vaciada")

✅ Base de Datos vaciada


---

# Day 3 (Context Engineering: Sessions & Memory) | PARTE 2

---

##  Sección 1: Configuración previa

Segunda parte del manual del dia 3.

Aqui usaremos Memory, no Sessions. La diferencia radica en que Sessions es temporal, persistencia muy comparable a una memoria RAM, y en cambio Memory es mas como una persistencia a largo plazo, imitando el conocimiento.

Ejemplo: Imagina que hablas con un asistente personal:

🗣️ Session: Recuerda lo que dijiste hace 10 minutos en ESTA conversación


🧠 Memory: Recuerda tus preferencias de conversaciones de LA SEMANA PASADA

> ⚠️ **Nota importante**
>
> Iniciamos primero
>
> 
>
> la configuración.

In [18]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [19]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [24]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

> ⚠️ **Nota importante**
>
> Fin de la configuración

##  Sección 2: Flujo de trabajo de Memoria

Desde la sección de Introducción, ya sabes por qué necesitamos Memoria. Para integrar Memoria en tus Agentes, hay tres pasos de alto nivel.

### Proceso de integración en tres pasos

1. **Inicializar**  
   → Crear un `MemoryService` y proporcionarlo a tu agente a través del `Runner`

2. **Ingerir**  
   → Transferir los datos de la sesión a la memoria usando `add_session_to_memory()`

3. **Recuperar**  
   → Buscar recuerdos almacenados usando `search_memory()`


---

#  Sección 3: Inicializar MemoryService

### 3.1 Inicializar Memoria

ADK proporciona múltiples implementaciones de `MemoryService` a través de la interfaz `BaseMemoryService`:

- **InMemoryMemoryService**  
  Servicio integrado para prototipado y pruebas  
  *(coincidencia por palabras clave, sin persistencia)*

- **VertexAiMemoryBankService**  
  Servicio gestionado en la nube con consolidación impulsada por LLM y búsqueda semántica

- **Implementaciones personalizadas**  
  Puedes crear la tuya propia usando bases de datos, aunque se recomiendan los servicios gestionados

Para este notebook, utilizaremos **InMemoryMemoryService** para aprender la mecánica central.  
Los mismos métodos funcionan de manera idéntica con servicios listos para producción como **Vertex AI Memory Bank**.


In [22]:
memory_service = (

    InMemoryMemoryService()

)

# Servicio de Memoria integrado de ADK para desarrollo y pruebas


### 3.2 Agregar memoria al agente

A continuación, crea un agente sencillo para responder a las consultas de los usuarios.



In [25]:
# Definimos constantes

APP_NAME = "DemoAppMemoria"
USER_NAME = "demo_user"

# Creación del agente

user_agent=LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
    name="MemoryAgentDemo",
    instruction="Contestar las preguntas del usuario con palabras simples",
)

print(f"Agente de nombre {user_agent.name} creado correctamente")



Agente de nombre MemoryAgentDemo creado correctamente


### Crear Runner

Ahora proporciona los servicios de **Session** y **Memory** al **Runner**.

#### Configuración clave

El Runner requiere ambos servicios para habilitar la funcionalidad de memoria:

- **session_service** → Gestiona los hilos de conversación y los eventos  
- **memory_service** → Proporciona almacenamiento de conocimiento a largo plazo  

Ambos servicios trabajan en conjunto: **Sessions** capturan las conversaciones y **Memory** almacena el conocimiento para su recuperación a través de distintas sesiones.


In [30]:
# Creando Session Service

session_service=InMemorySessionService() # -> Maneja las conversaciones

# Creando el runner con LOS DOS servicios

runner=Runner(
    agent=user_agent,
    app_name="DemoAppMemoria",
    session_service=session_service,
    memory_service=memory_service,
)

print("Agente y runner creados satisfactoriamente con soporte de memoria")



Agente y runner creados satisfactoriamente con soporte de memoria


‼️ Importante

💡 **Configuración vs. uso**: 

Agregar `memory_service` al Runner hace que la memoria esté disponible para tu agente, pero **no la utiliza automáticamente**. Debes hacerlo de forma explícita:

- Ingerir datos usando `add_session_to_memory()`
- Habilitar la recuperación dando a tu agente herramientas de memoria (`load_memory` o `preload_memory`)

¡Aprendamos estos pasos a continuación!


### 3.3 Opciones de implementación de MemoryService

**Este notebook:** `InMemoryMemoryService`

- Almacena eventos de conversación sin consolidación
- Búsqueda basada en palabras clave (coincidencia simple de palabras)
- Almacenamiento en memoria (se reinicia al reiniciar)
- Ideal para aprendizaje y desarrollo local

**Producción:** `VertexAiMemoryBankService` (lo aprenderás en el Día 5)

- Extracción de hechos clave impulsada por LLM
- Búsqueda semántica (recuperación basada en significado)
- Almacenamiento persistente en la nube
- Integra fuentes de conocimiento externas

💡 **Consistencia de la API**: Ambas implementaciones usan métodos idénticos (`add_session_to_memory()`, `search_memory()`). ¡El flujo de trabajo que aprendas aquí aplica a todos los servicios de memoria!


# 💾 **Sección 4: Ingestar datos de sesión en la memoria**

### ¿Por qué deberías transferir los datos de la sesión a la memoria?

Ahora que la memoria está inicializada, necesitas poblarla con conocimiento. Cuando inicializas un `MemoryService`, este comienza completamente vacío. Todas tus conversaciones se almacenan en **sesiones**, que contienen eventos sin procesar, incluyendo cada mensaje, llamada a herramientas y metadatos. Para que esta información esté disponible para el recuerdo a largo plazo, debes transferirla explícitamente a la memoria usando `add_session_to_memory()`.

Aquí es donde brillan los servicios de memoria gestionados como **Vertex AI Memory Bank**. Durante la transferencia, realizan una consolidación inteligente: extraen los hechos clave mientras descartan el ruido conversacional. El `InMemoryMemoryService` que estamos usando almacena todo sin consolidación, lo cual es suficiente para aprender la mecánica.

Antes de poder transferir cualquier cosa, necesitamos datos. Tengamos una conversación con nuestro agente para poblar la sesión. Esta conversación se almacenará en el `SessionService`, tal como aprendiste en el cuaderno anterior.


In [31]:


# Digaqmos al agente por ejemplo,nuestro color favorito
await run_session(
    runner,
    "Mi color favorito es el verde, como el chakra corazón. ¿puedes decirme algo al respecto?",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > Mi color favorito es el verde, como el chakra corazón. ¿puedes decirme algo al respecto?
Model: > ¡Claro que sí! El chakra del corazón se asocia con el color verde.

Se dice que el chakra del corazón es el centro del amor, la compasión y la conexión. Cuando está equilibrado, te sientes abierto, generoso y capaz de amar y ser amado.

Si quieres conectar más con tu chakra del corazón, puedes probar a:

*   **Meditación:** Enfócate en tu pecho y visualiza una luz verde brillante.
*   **Afirmaciones:** Repite frases como "Soy amor" o "Acepto y doy amor libremente".
*   **Actividades:** Pasa tiempo en la naturaleza, abraza a tus seres queridos o haz algo que te haga sentir feliz.

¡Espero que esto te ayude!


---







#### Verifiquemos que la conversación fue capturada en la sesión. Deberías ver los eventos de la sesión que contienen tanto el mensaje del usuario como la respuesta del modelo.







In [32]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Veamos qué hay en la sesión

print("📝 Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")

📝 Session contains:
  user: Mi color favorito es el verde, como el chakra corazón. ¿pued...
  model: ¡Claro que sí! El chakra del corazón se asocia con el color ...


#### ¡Perfecto! La sesión contiene nuestra conversación. Ahora estamos listos para transferirla a la memoria. Llama a add_session_to_memory() y pasa el objeto de sesión. Esto ingiere la conversación en el almacén de memoria, haciéndola disponible para búsquedas futuras.

In [33]:
# Este es el METODO CLAVE del asunto!

await memory_service.add_session_to_memory(session)

print("Session añadida a memoria")

Session añadida a memoria


# Sección 5: Habilitar la recuperación de memoria en tu agente

Has transferido correctamente los datos de la sesión a la memoria, pero todavía queda un paso crucial. Los agentes no pueden acceder directamente al `MemoryService`; necesitan **herramientas** para poder buscar en él.

Esto es intencional: te da control sobre **cuándo** y **cómo** se recupera la memoria.


### 5.1 Recuperación de memoria en ADK

ADK proporciona dos herramientas integradas para la recuperación de memoria:

#### `load_memory` (Reactivo)

- El agente decide cuándo buscar en la memoria  
- Solo recupera información cuando el agente considera que es necesario  
- Más eficiente (ahorra tokens)  
- Riesgo: el agente podría olvidar buscar en la memoria  

#### `preload_memory` (Proactivo)

- Busca automáticamente antes de cada turno  
- La memoria siempre está disponible para el agente  
- Contexto garantizado, pero menos eficiente  
- Realiza búsquedas incluso cuando no son necesarias  

Piénsalo como estudiar para un examen: `load_memory` es consultar tus apuntes solo cuando los necesitas, mientras que `preload_memory` es leer todos tus apuntes antes de responder cada pregunta.


### 5.2 Añadir la herramienta Load Memory al agente

Comencemos implementando el patrón reactivo. Volveremos a crear el agente de la Sección 3, esta vez añadiendo la herramienta `load_memory` a su conjunto de herramientas. Dado que se trata de una herramienta integrada de ADK, simplemente hay que incluirla en el arreglo de herramientas sin necesidad de ninguna implementación personalizada.


In [41]:
# Creando de nuevo el agente

user_agenT=LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
    name="DemoMemoryAgent",
    instruction="""
                    Puedes recordar información de conversaciones pasadas usando la herramienta load_memory.
                    Si el usuario pregunta algo que pudo haber dicho antes, busca siempre en la memoria primero.
                    Si encuentras información relevante, úsala para responder.

                """,
    tools=[load_memory], # ¡Ahora el agente tiene acceso a la memoria y puede buscar en ella cuando lo considere necesario!
)

print(f"Agente de nombre {user_agent.name} con la herramienta load_memory creado satisfactoriamente")

Agente de nombre MemoryAgentDemo con la herramienta load_memory creado satisfactoriamente


### 5.3 Actualizar el Runner y probar

Ahora actualicemos el `Runner` para usar nuestro nuevo `user_agent`, que ya cuenta con la herramienta `load_memory`. Luego le preguntaremos al agente sobre el color favorito que habíamos almacenado previamente en otra sesión.

👉 Dado que las sesiones no comparten el historial de conversación, la única forma en que el agente puede responder correctamente es usando la herramienta `load_memory` para recuperar la información desde la memoria a largo plazo que almacenamos manualmente.


In [43]:
# Creando un nuevo Runner con la nueva actualización del Agente

runner=Runner(
    agent=user_agent,
    app_name="DemoAppMemoria",
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "Cual es mi color favorito", "color-test")


### Session: color-test

User > Cual es mi color favorito
Model: > No tengo forma de saber cuál es tu color favorito. No recuerdo información personal sobre ti.
